In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np
from sklearn.model_selection import train_test_split

/Users/varunnegandhi/anaconda3/envs/tensorflow-sessions/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/varunnegandhi/anaconda3/envs/tensorflow-sessions/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/varunnegandhi/anaconda3/envs/tensorflow-sessions/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16"

In [2]:
train = pd.read_csv("Training_Data.csv")
print(train)
test = pd.read_csv("Test_Data.csv",encoding="latin-1")
test['Sentiment']=test["Sentiment"].map({1:'positive' , 0: 'negative'})
print(test)
#data = pd.concat([train,test])
data = train
print(data)

                                           SentimentText Sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]
                                           SentimentText Sentiment
0      first think another Disney movie, might good, ...  positive
1      Put aside Dr. House repeat mi

In [3]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" 
embed = hub.Module(module_url)

In [4]:
sentence_list = list(data['SentimentText'])
print(sentence_list)

with tf.Session() as session:
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  sentence_embeddings = session.run(embed(sentence_list))

print(np.array(sentence_embeddings))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


[[ 0.03087823 -0.06816969  0.00478594 ... -0.04904244  0.00312513
   0.04500568]
 [ 0.024796   -0.03751242 -0.00023623 ... -0.01052065 -0.01016857
  -0.01321065]
 [ 0.02137806  0.05823477 -0.01021961 ...  0.06828433 -0.01064305
   0.072465  ]
 ...
 [-0.0076001   0.01783323 -0.00141936 ... -0.05320873 -0.02816549
   0.06544039]
 [-0.01944417 -0.03632438 -0.00631774 ... -0.02648013  0.00287605
   0.03750989]
 [ 0.06568789  0.01749829 -0.055237   ... -0.02553775 -0.04536348
   0.01368993]]


In [5]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(np.array(sentence_embeddings))
similarity.shape

(50000, 50000)

In [44]:
col_names = list(data.columns)
col_names
df = pd.DataFrame(columns = col_names)
print(df)

neighbors = []
print(len(sentence_list))
#for n in range(len(sentence_list)):
    #print(n)
sentence = sentence_list[5]
i = sentence_list.index(sentence)
row = np.array(similarity[i,:])
print(row)
sentence_index = row.argsort()[-2:][::-1]
print(sentence_embeddings[sentence_index])
print(sentence_embeddings[sentence_index].shape)
print(sentence_index)
print(row[sentence_index])
print(*[sentence_list[i] for i in sentence_index], sep="\n")
neighbors.append(sentence_embeddings[sentence_index])
print("Required matrix:")
print(neighbors)
print("Shape:", np.array(neighbors).shape)
#print(df)

Empty DataFrame
Columns: [SentimentText, Sentiment]
Index: []
50000
[0.30699748 0.50442725 0.65838945 ... 0.49659336 0.64077044 0.6159339 ]
[[-0.02724895  0.00743809 -0.00618643 ... -0.00343892  0.00357993
   0.03877556]
 [ 0.01704155  0.04490056 -0.00240944 ... -0.03973788 -0.01330681
   0.00876388]]
(2, 512)
[    5 30495]
[0.9999999 0.8495669]
Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it's not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like "dressed-up midgets" than children, but that only makes them more fun to watch. And the mother's slow awakening to what's happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they'd all be "up" for this mo

In [8]:
data2 = pd.DataFrame([neighbors]).transpose()
print(data.shape)
print(data2.shape)
data2['SentimentText']=data['SentimentText']
data2['Sentiment']=data['Sentiment']
print(data2.shape)
data2.rename(columns = {0:'Embedding_Matrix'}, inplace=True)
print(data2)

(50000, 2)
(50000, 1)
(50000, 3)
                                        Embedding_Matrix  \
0      [[0.030878235, -0.06816969, 0.004785944, -0.05...   
1      [[0.024796002, -0.037512418, -0.0002362301, -0...   
2      [[0.021378059, 0.05823477, -0.010219607, -0.06...   
3      [[-0.0059442, -0.057756796, -0.03130618, -0.02...   
4      [[0.050902136, 0.04202257, -0.014997388, -0.06...   
...                                                  ...   
49995  [[-0.0045723096, 0.015551986, -0.022343015, -0...   
49996  [[0.019559477, 0.0016787908, -0.015686644, -0....   
49997  [[-0.007600099, 0.017833227, -0.0014193594, 0....   
49998  [[-0.019444171, -0.03632438, -0.006317741, -0....   
49999  [[0.065687895, 0.017498286, -0.055237003, -0.0...   

                                           SentimentText Sentiment  
0      One of the other reviewers has mentioned that ...  positive  
1      A wonderful little production. <br /><br />The...  positive  
2      I thought this was a wonderful w

In [9]:
y = data2['Sentiment']
le = preprocessing.LabelEncoder()
le.fit(y)

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

y_enc = encode(le, y)

In [10]:
print(data2['Embedding_Matrix'][0].shape)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data2['Embedding_Matrix'], y_enc, test_size=0.33, random_state=42)
print(x_train)
print(x_train.shape)

(25, 512)
23990    [[-0.043813415, -0.0053389403, -0.032010645, -...
8729     [[0.018371582, -0.011922709, 0.0033760297, -0....
3451     [[-0.010972981, -0.01482286, -0.00017860503, -...
2628     [[0.022037432, 0.030444149, -0.008778872, -0.0...
38352    [[0.05383027, 0.024601217, -0.06429972, -0.040...
                               ...                        
11284    [[0.05198604, -0.021432154, -0.0005987687, -0....
44732    [[-0.009684506, -0.03869877, -0.032634277, -0....
38158    [[-0.0022769442, -0.03521671, -0.031315487, -0...
860      [[-0.00472308, -0.01658315, 0.02326638, -0.080...
15795    [[0.057155646, -0.04065958, -0.044187304, -0.0...
Name: Embedding_Matrix, Length: 33500, dtype: object
(33500,)


In [40]:
import keras
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Dense, Flatten
model = keras.models.Sequential()
model.add(Convolution2D(filters=32,kernel_size=3,input_shape=(25,512,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(filters=64,kernel_size=3))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(filters=80,kernel_size=3))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 23, 510, 32)       320       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 11, 255, 32)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 9, 253, 64)        18496     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 4, 126, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 2, 124, 80)        46160     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 1, 62, 80)         0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 1, 62, 80)         0         
__________

In [41]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [42]:
x_train = np.array(list(x_train))
x_train = x_train.reshape([-1,25, 512,1])
#print(x_train)
print(x_train.shape)
print(x_train[860][0].shape)
model.fit(x_train, y_train, 
          batch_size=1000, epochs=10, verbose=1, validation_split=0.2)

(33500, 25, 512, 1)
(512, 1)
Train on 26800 samples, validate on 6700 samples
Epoch 1/10
26800/26800 [==============================] - 544s 20ms/step - loss: 0.5638 - acc: 0.7042 - val_loss: 0.4478 - val_acc: 0.8007
Epoch 2/10
26800/26800 [==============================] - 544s 20ms/step - loss: 0.4412 - acc: 0.7973 - val_loss: 0.4165 - val_acc: 0.8143
Epoch 3/10
26800/26800 [==============================] - 510s 19ms/step - loss: 0.4225 - acc: 0.8096 - val_loss: 0.4078 - val_acc: 0.8200
Epoch 4/10
26800/26800 [==============================] - 528s 20ms/step - loss: 0.4113 - acc: 0.8153 - val_loss: 0.3968 - val_acc: 0.8234
Epoch 5/10
26800/26800 [==============================] - 524s 20ms/step - loss: 0.4064 - acc: 0.8170 - val_loss: 0.3925 - val_acc: 0.8237
Epoch 6/10
26800/26800 [==============================] - 551s 21ms/step - loss: 0.3961 - acc: 0.8219 - val_loss: 0.3912 - val_acc: 0.8281
Epoch 7/10
26800/26800 [==============================] - 541s 20ms/step - loss: 0.3901 

In [32]:
x_test = np.array(list(x_test))
x_test = x_test.reshape([-1,25, 512,1])
#print(x_train)
print(x_test.shape)

(16500, 25, 512, 1)


In [43]:
predicts = model.predict(x_test, batch_size=1000)
print(predicts)

[[0.48863938 0.5113605 ]
 [0.04703605 0.95296395]
 [0.8830941  0.11690585]
 ...
 [0.01500397 0.98499596]
 [0.09716868 0.9028314 ]
 [0.7744955  0.2255045 ]]


In [44]:
y_test1 = decode(le, y_test)
y_preds = decode(le, predicts)
print(y_preds)
print(y_test1.dtype)

from sklearn import metrics
metrics.confusion_matrix(list(y_test1), list(y_preds))
print(metrics.classification_report(list(y_test1), list(y_preds)))

['positive' 'positive' 'negative' ... 'positive' 'positive' 'negative']
object
              precision    recall  f1-score   support

    negative       0.86      0.79      0.82      8208
    positive       0.81      0.88      0.84      8292

    accuracy                           0.83     16500
   macro avg       0.84      0.83      0.83     16500
weighted avg       0.84      0.83      0.83     16500



In [77]:
new_data = pd.DataFrame(sentence_embeddings)
new_data['Sentiment']=data['Sentiment']
new_data
X_train1, 

AttributeError: 'DataFrame' object has no attribute 'map'

In [13]:
df = pd.read_csv("Amazon-Product-Reviews.csv")
df.groupby('reviews.rating').count()[[]]

,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
reviews.rating,,,,,,,,,,,,,,,,,,,,
1.0,410,326,410,410,410,410,410,410,269,410,0,356,0,375,410,410,410,0,0,410
2.0,402,339,402,402,402,402,402,402,280,402,0,382,0,388,402,402,402,0,0,402
3.0,1499,1289,1499,1499,1499,1499,1499,1499,1081,1499,0,1470,0,1475,1499,1499,1499,0,0,1499
4.0,8541,7220,8540,8541,8541,8541,8541,8531,6191,8541,0,8469,0,8480,8541,8541,8541,0,0,8541
5.0,23775,18694,23774,23775,23775,23775,23775,23756,16218,23775,1,23389,1,23413,23775,23773,23770,0,0,23773
